<a href="https://colab.research.google.com/github/gizdatalab/CPU/blob/main/classifier/SECTOR/CPU_SECTOR_SentenceT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! pip install datasets transformers sentencepiece huggingface_hub
! apt install git-lfs
! pip install sentence-transformers
! pip install optuna
! pip install evaluate
! pip install accelerate -U
! pip install codecarbon

In [ ]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 21:33:29] [setup] RAM Tracking...
[codecarbon INFO @ 21:33:29] [setup] GPU Tracking...
[codecarbon INFO @ 21:33:29] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 21:33:29] [setup] CPU Tracking...
[codecarbon WARNING @ 21:33:29] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 21:33:31] We saw that you have a AMD EPYC 7V12 64-Core Processor but we don't know it. Please contact us.
[codecarbon INFO @ 21:33:31] CPU Model on constant consumption mode: AMD EPYC 7V12 64-Core Processor
[codecarbon INFO @ 21:33:31] >>> Tracker's metadata:
[codecarbon INFO @ 21:33:31]   Platform system: Linux-5.15.0-1040-azure-x86_64-with-glibc2.10
[codecarbon INFO @ 21:33:31]   Python version: 3.8.5
[codecarbon INFO @ 21:33:31]   CodeCarbon version: 2.3.2
[codecarbon INFO @ 21:33:31]   Available RAM : 27.406 GB
[codecarbon INFO @ 21:33:31]   CPU count: 4
[codecarbon INFO @ 21:33:31]   CPU model: AMD EPYC 7V12 64-Core Processor
[codecarbon INFO @ 21:33:

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

[codecarbon INFO @ 21:33:46] Energy consumed for RAM : 0.000043 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 21:33:46] Energy consumed for all GPUs : 0.000108 kWh. Total GPU Power : 25.905571814272516 W
[codecarbon INFO @ 21:33:46] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:33:46] 0.000328 kWh of electricity used since the beginning.
[codecarbon INFO @ 21:34:01] Energy consumed for RAM : 0.000086 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 21:34:01] Energy consumed for all GPUs : 0.000215 kWh. Total GPU Power : 25.7171847747454 W
[codecarbon INFO @ 21:34:01] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:34:01] 0.000655 kWh of electricity used since the beginning.
No such comm: 35c1757231de4a10bfaedb122bd98ed7
[codecarbon INFO @ 21:34:16] Energy consumed for RAM : 0.000128 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 21:34:16] Energy consumed for all GPUs : 0.000

In [ ]:
from datasets import load_dataset
import datasets
import numpy as np
import pandas as pd
import evaluate
import torch
import os
import sklearn.metrics as skm
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          RobertaForSequenceClassification,
                          TrainingArguments, Trainer, TrainerCallback)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

2024-01-21 21:34:02.541988: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-21 21:34:02.656872: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-21 21:34:02.656892: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-01-21 21:34:03.277046: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [ ]:
# Load Dataset from Hugging Face
try:
    dataset = load_dataset("GIZ/policy_classification", data_files="policy_classification.json",use_auth_token=True )
    data = pd.DataFrame(dataset['train'])
except:
    from huggingface_hub import hf_hub_download
    hf_hub_download("GIZ/policy_classification", filename="policy_classification.json", repo_type="dataset",use_auth_token=True,
    local_dir= 'data/')
    data = pd.read_json('data/policy_classification.json')
data = data.dropna(subset = ['SectorLabel']).reset_index(drop=True)
sector_list = [val  for list_ in list(data.SectorLabel) for val in list_]
sector_list = list(set(sector_list))

for sector in sector_list:
    data[sector] = data.SectorLabel.apply(lambda x: True if sector in x else False)

Using custom data configuration GIZ--policy_classification-cdc4b717b1bd720b


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Failed to read file '/home/azureuser/.cache/huggingface/datasets/downloads/dee24a3afc63cd87c4a0a9517165e43822030f22df905fa089a2628e8d786ca4' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Column() changed from object to array in row 0


# Manual Inputs


In [ ]:
###### Define Path to data ############################
repo_id = "GIZ/policy_classification"
filename="policy_classification.json"


### Define labels ##################
label_names= sector_list


#######  Define Model params ######################

# define the model checkpoint
model_checkpoint = "sentence-transformers/all-mpnet-base-v2"

# define the problem type
problem_type="multi_label_classification"


######## Training Arguments ##################
output_dir = "SECTOR-multilabel-mpnet_w"

# logging and evaluation
evaluation_strategy = "epoch"
logging_strategy="epoch"
logging_steps = 1
eval_steps = 1

# Important params
learning_rate=7.04e-5   #4.04e-5
batch_size = 8
num_train_epochs= 10
weight_decay=0.095
warmup_steps = 300
gradient_accumulation_steps = 2

# Automated

## Dataset

In [ ]:
# Load Dataset from Hugging Face
# Load Dataset from Hugging Face
# try:
#     dataset = load_dataset(repo_id, data_files=filename,
#                              use_auth_token=True )
#     data = pd.DataFrame(dataset['train'])
# except:
#     from huggingface_hub import hf_hub_download
#     hf_hub_download(repo_id, filename=filename,
#                     repo_type="dataset",use_auth_token=True,
#     local_dir= 'data/')
#     data = pd.read_json('data/policy_classification.json')

# the dataset already contains train_test split flag we use
# it to create train-test split.

train_df = data[data.split== 'Train'].reset_index(drop=True)
test_df = data[data.split=='Test'].reset_index(drop=True)

# Sort the labels and create plaecholders for id2label and label2id
label_names.sort()

num_labels = len(label_names)
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {label:idx for idx, label in enumerate(label_names)}
print(id2label)

{0: 'Agriculture', 1: 'Buildings', 2: 'Coastal Zone', 3: 'Cross-Cutting Area', 4: 'Disaster Risk Management (DRM)', 5: 'Economy-wide', 6: 'Education', 7: 'Energy', 8: 'Environment', 9: 'Health', 10: 'Industries', 11: 'LULUCF/Forestry', 12: 'Social Development', 13: 'Tourism', 14: 'Transport', 15: 'Urban', 16: 'Waste', 17: 'Water'}


In [ ]:
def prepare_multilabel_df(df, cols):
    """
    Takes the dataframe and list of columns which need to combined into
    One-hot-encoded labels.

    """
    df.dropna(subset = cols, inplace=True)
    df = df.reset_index(drop=True)
    df['labels'] = df.apply(lambda x: [int(x[label]) for label in cols],axis=1)
    # we have multilingual dataset ('en', 'fr','es'), in case of non-english the
    # translated Context is udner the column 'TranslatedContext'. We impute these
    # back into 'context' to work with only English dataset.
    df['Context'] = df.apply(lambda x: x['Context'] if x['Language'] == 'en'
                                    else x['TranslatedContext'][0],axis=1)
    df.rename(columns = {'Context':'text'}, inplace=True)

    df = df[['text','labels']+cols]
    return df

# creating multilabel dataset
train_df = prepare_multilabel_df(train_df, label_names)
test_df = prepare_multilabel_df(test_df, label_names)

print('\n','Classes Representation in Training Dataset:',len(train_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(train_df['labels'])]))

print('\n','Classes Representation in Test Dataset:',len(test_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(test_df['labels'])]))


 Classes Representation in Training Dataset: 10123
0 . Agriculture : 2235
1 . Buildings : 169
2 . Coastal Zone : 698
3 . Cross-Cutting Area : 1853
4 . Disaster Risk Management (DRM) : 814
5 . Economy-wide : 873
6 . Education : 180
7 . Energy : 2847
8 . Environment : 905
9 . Health : 662
10 . Industries : 419
11 . LULUCF/Forestry : 1861
12 . Social Development : 507
13 . Tourism : 192
14 . Transport : 1173
15 . Urban : 558
16 . Waste : 714
17 . Water : 1207

 Classes Representation in Test Dataset: 1125
0 . Agriculture : 236
1 . Buildings : 21
2 . Coastal Zone : 78
3 . Cross-Cutting Area : 204
4 . Disaster Risk Management (DRM) : 92
5 . Economy-wide : 100
6 . Education : 27
7 . Energy : 315
8 . Environment : 95
9 . Health : 76
10 . Industries : 51
11 . LULUCF/Forestry : 215
12 . Social Development : 70
13 . Tourism : 28
14 . Transport : 133
15 . Urban : 61
16 . Waste : 73
17 . Water : 124


[codecarbon INFO @ 21:35:46] Energy consumed for RAM : 0.000385 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 21:35:46] Energy consumed for all GPUs : 0.000965 kWh. Total GPU Power : 25.78838454547421 W
[codecarbon INFO @ 21:35:46] Energy consumed for all CPUs : 0.001594 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:35:46] 0.002944 kWh of electricity used since the beginning.


In [ ]:
for i,label in enumerate(label_names):
    # df_train[sector] = df_train.apply(lambda x: x['sector_label'][i], axis =1)
    print(i,".",label, ":", sum(train_df[label]))

# these are weights which can be used in building weighted loss if class is imbalanced
positive_weights = {}
negative_weights = {}
for label in label_names:
    positive_weights[label] = train_df.shape[0]/(2*np.count_nonzero(train_df[label]==True))
    negative_weights[label] = train_df.shape[0]/(2*np.count_nonzero(train_df[label]==False))
print("Positive Weights",'\n',positive_weights)
print("Negative Weights",'\n',negative_weights)

# as we dont want to miss anything (high recall) we use positive weights calculated above.
pos_weights = list(positive_weights.values())

# If using GPU we need to place all required data on else there will be error
posweights = torch.FloatTensor(pos_weights).to(device)

0 . Agriculture : 2235
1 . Buildings : 169
2 . Coastal Zone : 698
3 . Cross-Cutting Area : 1853
4 . Disaster Risk Management (DRM) : 814
5 . Economy-wide : 873
6 . Education : 180
7 . Energy : 2847
8 . Environment : 905
9 . Health : 662
10 . Industries : 419
11 . LULUCF/Forestry : 1861
12 . Social Development : 507
13 . Tourism : 192
14 . Transport : 1173
15 . Urban : 558
16 . Waste : 714
17 . Water : 1207
Positive Weights 
 {'Agriculture': 2.264653243847875, 'Buildings': 29.949704142011836, 'Coastal Zone': 7.251432664756447, 'Cross-Cutting Area': 2.731516459794927, 'Disaster Risk Management (DRM)': 6.218058968058968, 'Economy-wide': 5.797823596792669, 'Education': 28.119444444444444, 'Energy': 1.7778363189322093, 'Environment': 5.592817679558011, 'Health': 7.645770392749244, 'Industries': 12.079952267303103, 'LULUCF/Forestry': 2.7197743148844706, 'Social Development': 9.983234714003945, 'Tourism': 26.361979166666668, 'Transport': 4.315004262574595, 'Urban': 9.07078853046595, 'Waste': 

`pos_weight > 1 will increase the recall while pos_weight < 1 will increase the precision.`

## Model

In [ ]:
# problem_type, is not needed in tokenizer but keeping it for conformity
# https://huggingface.co/docs/transformers/main_classes/configuration?highlight=multi_label_classification#transformers.PretrainedConfig
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,problem_type=problem_type)

# Set the model config and assign it and device to model
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id, problem_type=problem_type)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,config=config).to(device)

Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# we need datasets format to work with
train_ds = datasets.Dataset.from_pandas(train_df[['text','labels']])
test_ds =  datasets.Dataset.from_pandas(test_df[['text','labels']])
train_ds = train_ds.shuffle(seed=7)
test_ds = test_ds.shuffle(seed=7)


# Need to tokenize the data using the tokenizer of the model
def tokenize_and_encode(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,
                        max_length=384)

cols = train_ds.column_names
cols.remove("labels")
print('Training data:',train_ds.num_rows)
print('Validation data:',test_ds.num_rows)

train_tokenized = train_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)
val_tokenized = test_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)

# need this to avoid error due to type mismatch
# https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3
train_tokenized.set_format("torch")
train_tokenized = (train_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

val_tokenized.set_format("torch")
val_tokenized = (val_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

Parameter 'function'=<function tokenize_and_encode at 0x7f8fad7ec550> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Training data: 10123
Validation data: 1125


  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

[codecarbon INFO @ 21:36:01] Energy consumed for RAM : 0.000428 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 21:36:01] Energy consumed for all GPUs : 0.001072 kWh. Total GPU Power : 25.60944052969043 W
[codecarbon INFO @ 21:36:01] Energy consumed for all CPUs : 0.001771 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:36:01] 0.003271 kWh of electricity used since the beginning.


  0%|          | 0/10123 [00:00<?, ?ex/s]

  0%|          | 0/1125 [00:00<?, ?ex/s]

## Metrics

In [ ]:
multilabel_f1_metric = evaluate.load("f1", "multilabel")
multilabel_recall_metric = evaluate.load("recall","multilabel")
multilabel_precision_metric = evaluate.load("precision","multilabel")

def get_scores(y_pred, y_test, thresh=0.5, sigmoid=True):
    """
    Takes the Predictions and Truth Values, and return multilabel metrics
    aggregated on multiple strategies['micro', 'sample', 'weighted']

    """
    # we get raw function value, therefore we apply sigmoid to get to the
    # probabilities
    if problem_type == "multi_label_classification":
        y_pred = torch.from_numpy(y_pred)
        y_test = torch.from_numpy(y_test)
        if sigmoid:
          y_pred = y_pred.sigmoid()
          y_pred = (y_pred > thresh)

        return {
            "precision-micro": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="micro")["precision"],
            "precision-samples": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="samples")["precision"],
            "precision-weighted": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="weighted")["precision"],
            "recall-micro": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="micro")["recall"],
            "recall-samples": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="samples")["recall"],
            "recall-weighted": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="weighted")["recall"],
            "f1-micro": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"],
            "f1-samples": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="samples")["f1"],
            "f1-weighted": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="weighted")["f1"],
            }
    else:
        y_pred = torch.from_numpy(y_pred).argmax(dim=1)
        y_test = torch.from_numpy(y_test)

        report = skm.classification_report(y_test, y_pred, output_dict=True)
        df_report = pd.DataFrame(report).transpose()
        return {"Precision_macro": df_report.loc['macro avg']['precision'],
                "Precision_weighted": df_report.loc['weighted avg']['precision'],
                "Recall_macro": df_report.loc['macro avg']['recall'],
                "Recall_weighted": df_report.loc['weighted avg']['recall'],
                "F1-Score": df_report.loc['macro avg']['f1-score'],
                "Accuracy": skm.accuracy_score(y_test, y_pred)}


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return get_scores(predictions, labels)

[codecarbon INFO @ 21:36:16] Energy consumed for RAM : 0.000471 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 21:36:16] Energy consumed for all GPUs : 0.001178 kWh. Total GPU Power : 25.62088603560488 W
[codecarbon INFO @ 21:36:16] Energy consumed for all CPUs : 0.001948 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:36:16] 0.003597 kWh of electricity used since the beginning.


## Training arguments

In [ ]:
args = TrainingArguments(
    output_dir,
    logging_strategy=logging_strategy,
    evaluation_strategy = evaluation_strategy,
    save_strategy = "epoch",
    logging_steps = logging_steps,
    eval_steps= eval_steps,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    lr_scheduler_type = "linear",
    gradient_accumulation_steps = gradient_accumulation_steps,
    warmup_steps = warmup_steps,
    push_to_hub=True,
)

## Custom Trainer

In [ ]:
# for class weights we need to use Custom Multi-Label Trainer
# In multi-label problem we will be using Binary Cross Entropy loss with
# sigmoid layer on top rather than softmax.
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=posweights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels),
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss


# Training

In [ ]:
multi_trainer =  MultilabelTrainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)


[codecarbon INFO @ 21:36:31] [setup] RAM Tracking...
[codecarbon INFO @ 21:36:31] [setup] GPU Tracking...
[codecarbon INFO @ 21:36:31] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 21:36:31] [setup] CPU Tracking...
[codecarbon WARNING @ 21:36:31] No CPU tracking mode found. Falling back on CPU constant mode.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[codecarbon INFO @ 21:36:31] Energy consumed for RAM : 0.000514 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 21:36:31] Energy consumed for all GPUs : 0.001285 kWh. Total GPU Power : 25.709045558985192 W
[codecarbon INFO @ 21:36:31] Energy consumed for all CPUs : 0.002125 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:36:31] 0.003924 kWh of elect

In [ ]:
multi_trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
You're using a MPNetTokenize

Epoch,Training Loss,Validation Loss,Precision-micro,Precision-samples,Precision-weighted,Recall-micro,Recall-samples,Recall-weighted,F1-micro,F1-samples,F1-weighted
1,0.727300,0.573931,0.467650,0.583387,0.527781,0.697849,0.792872,0.697849,0.560016,0.618395,0.584316
2,0.493000,0.515846,0.584330,0.679004,0.623267,0.708854,0.812336,0.708854,0.640597,0.695206,0.652600
3,0.400100,0.428119,0.523031,0.653402,0.591138,0.783892,0.854773,0.783892,0.627427,0.695573,0.652617
4,0.318000,0.423837,0.579715,0.692366,0.610076,0.794897,0.870562,0.794897,0.670464,0.729940,0.683519
5,0.249800,0.420427,0.595681,0.713838,0.617407,0.800400,0.878157,0.800400,0.683031,0.746343,0.691971
6,0.194000,0.466918,0.595903,0.731542,0.619977,0.800400,0.872424,0.800400,0.683177,0.755018,0.693612
7,0.148800,0.458640,0.653421,0.764176,0.674825,0.797899,0.874256,0.797899,0.718468,0.779955,0.726274
8,0.116600,0.474802,0.656402,0.769541,0.667261,0.810405,0.881422,0.810405,0.725319,0.785985,0.729821
9,0.089900,0.497459,0.682800,0.781895,0.691177,0.790395,0.864750,0.790395,0.732669,0.787396,0.735754
10,0.074500,0.504309,0.686393,0.786448,0.693560,0.794897,0.870123,0.794897,0.736671,0.792881,0.739731


[codecarbon INFO @ 21:37:31] Energy consumed for RAM : 0.000685 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 21:37:31] Energy consumed for all GPUs : 0.001805 kWh. Total GPU Power : 47.902589582221694 W
[codecarbon INFO @ 21:37:31] Energy consumed for all CPUs : 0.002833 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:37:31] 0.005323 kWh of electricity used since the beginning.
[codecarbon INFO @ 21:37:38] Energy consumed for RAM : 0.000043 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 21:37:38] Energy consumed for all GPUs : 0.000285 kWh. Total GPU Power : 68.28096574573387 W
[codecarbon INFO @ 21:37:38] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:37:38] 0.000505 kWh of electricity used since the beginning.
[codecarbon INFO @ 21:37:46] Energy consumed for RAM : 0.000728 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 21:37:46] Energy consumed for all GPUs : 0.002094 kWh. Total GPU Power : 69.28940526017561 W

TrainOutput(global_step=6330, training_loss=0.2812098763753641, metrics={'train_runtime': 8214.6482, 'train_samples_per_second': 12.323, 'train_steps_per_second': 0.771, 'total_flos': 1.997891881072128e+16, 'train_loss': 0.2812098763753641, 'epoch': 10.0})

In [ ]:
tracker.stop()

[codecarbon INFO @ 23:54:38] Energy consumed for RAM : 0.024163 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 23:54:38] Energy consumed for all GPUs : 0.146845 kWh. Total GPU Power : 26.818382662092368 W
[codecarbon INFO @ 23:54:38] Energy consumed for all CPUs : 0.099956 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:54:38] 0.270963 kWh of electricity used since the beginning.


0.09903088036154775

In [ ]:
multi_trainer.push_to_hub()

'https://huggingface.co/ppsingh/SECTOR-multilabel-mpnet_w/tree/main/'

In [ ]:
log_results = multi_trainer.state.log_history
log_results

[{'loss': 0.7273,
  'learning_rate': 6.651223880597016e-05,
  'epoch': 1.0,
  'step': 633},
 {'eval_loss': 0.5739307999610901,
  'eval_precision-micro': 0.46765001676164936,
  'eval_precision-samples': 0.5833866923200257,
  'eval_precision-weighted': 0.5277812809936031,
  'eval_recall-micro': 0.6978489244622311,
  'eval_recall-samples': 0.7928716986716986,
  'eval_recall-weighted': 0.6978489244622311,
  'eval_f1-micro': 0.5600160578081091,
  'eval_f1-samples': 0.6183952960619628,
  'eval_f1-weighted': 0.5843155646376028,
  'eval_runtime': 25.5953,
  'eval_samples_per_second': 43.953,
  'eval_steps_per_second': 5.509,
  'epoch': 1.0,
  'step': 633},
 {'loss': 0.493,
  'learning_rate': 5.9121990049751246e-05,
  'epoch': 2.0,
  'step': 1266},
 {'eval_loss': 0.5158460140228271,
  'eval_precision-micro': 0.5843298969072165,
  'eval_precision-samples': 0.6790035273368606,
  'eval_precision-weighted': 0.6232674383219394,
  'eval_recall-micro': 0.7088544272136068,
  'eval_recall-samples': 0.81

In [ ]:
import json
with open("classifier_info/sector_mpnet_logs_w.json", "w") as fp:
    json.dump(log_results , fp)

# Predict and Evaluate

## Trainer

In [ ]:
predictions= multi_trainer.predict(val_tokenized)
pred,labels,_ = predictions
y_pred = torch.from_numpy(pred)
y_true = torch.from_numpy(labels)
y_prob = y_pred.sigmoid()
thresh = 0.5
y_pred = (y_prob>=thresh).bool()

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

Agriculture
[[833  56]
 [ 30 206]] 

Buildings
[[1092   12]
 [   5   16]] 

Coastal Zone
[[1010   37]
 [  23   55]] 

Cross-Cutting Area
[[815 106]
 [ 54 150]] 

Disaster Risk Management (DRM)
[[994  39]
 [ 27  65]] 

Economy-wide
[[957  68]
 [ 41  59]] 

Education
[[1086   12]
 [   7   20]] 

Energy
[[758  52]
 [ 38 277]] 

Environment
[[981  49]
 [ 24  71]] 

Health
[[1026   23]
 [  14   62]] 

Industries
[[1058   16]
 [   4   47]] 

LULUCF/Forestry
[[852  58]
 [ 34 181]] 

Social Development
[[1035   20]
 [  18   52]] 

Tourism
[[1085   12]
 [   9   19]] 

Transport
[[951  41]
 [ 27 106]] 

Urban
[[1021   43]
 [  20   41]] 

Waste
[[1023   29]
 [  10   63]] 

Water
[[948  53]
 [ 25  99]] 



In [ ]:
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,Agriculture,0.786260,0.872881,0.827309,236.0
1,Buildings,0.571429,0.761905,0.653061,21.0
2,Coastal Zone,0.597826,0.705128,0.647059,78.0
3,Cross-Cutting Area,0.585938,0.735294,0.652174,204.0
4,Disaster Risk Management (DRM),0.625000,0.706522,0.663265,92.0
5,Economy-wide,0.464567,0.590000,0.519824,100.0
6,Education,0.625000,0.740741,0.677966,27.0
7,Energy,0.841945,0.879365,0.860248,315.0
8,Environment,0.591667,0.747368,0.660465,95.0
9,Health,0.729412,0.815789,0.770186,76.0


## Pipeline

In [ ]:
from transformers import pipeline
model_checkpoint = "ppsingh/TAPP-multilabel-climatebert"
pipe = pipeline("text-classification", model=model_checkpoint, top_k=None, device=device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
predictions = pipe(list(test_df['text']))
pred = []
for prediction in predictions:
    pred.append({x['label']:x['score'] for x in prediction})
df_pred = pd.DataFrame(pred)
df_pred['labels'] = df_pred.apply(lambda x: np.array([x[label]
                            for label in label_names]) >= 0.50,axis=1)
y_true = np.array(list(test_df['labels']))
y_pred = np.array(list(df_pred['labels']))

In [ ]:
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

ActionLabel
[[417 104]
 [ 69 531]] 

PlansLabel
[[807  77]
 [ 71 166]] 

PolicyLabel
[[939  30]
 [ 28 124]] 

TargetLabel
[[737  62]
 [ 37 285]] 



In [ ]:
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,ActionLabel,0.836220,0.885000,0.859919,600.0
1,PlansLabel,0.683128,0.700422,0.691667,237.0
2,PolicyLabel,0.805195,0.815789,0.810458,152.0
3,TargetLabel,0.821326,0.885093,0.852018,322.0
4,micro avg,0.802030,0.843631,0.822305,1311.0
5,macro avg,0.786467,0.821576,0.803515,1311.0
6,weighted avg,0.801289,0.843631,0.821827,1311.0
7,samples avg,0.787021,0.808802,0.782698,1311.0
